In [ ]:
import pandas as pd
import numpy as np
import csv
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import xgboost as xgb
import scipy
from geopy import distance
import geopy
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn import preprocessing


%matplotlib inline
sns.set()
plt.rcParams['figure.figsize'] = [16, 10]

#### Read Data

In [ ]:
PATH_TRAIN_DATASET = './data/train.csv'
PATH_TEST_DATASET = './data/test.csv'
PATH_SAMPLE_SUMBISSION = './data/sample_submission.csv'

In [ ]:
df_test = pd.read_csv(PATH_TEST_DATASET, infer_datetime_format=True, parse_dates=['pickup_datetime'],  index_col='id')
df_train = pd.read_csv(PATH_TRAIN_DATASET, infer_datetime_format=True,parse_dates=['pickup_datetime'], index_col='id')
df_sample_submission = pd.read_csv(PATH_SAMPLE_SUMBISSION)
df_original_train = df_train.copy()
df_original_test = df_test.copy()

#### A brief description of the data

As mentioned in the challenge's page, the provided training dataset contains the following fields, as can be verified bellow:

- `id` - a unique identifier for each trip
- `vendor_id` - a code indicating the provider associated with the trip record
- `pickup_datetime` - date and time when the meter was engaged
- `dropoff_datetime` - date and time when the meter was disengaged
- `passenger_count` - the number of passengers in the vehicle (driver entered value)
- `pickup_longitude` - the longitude where the meter was engaged
- `pickup_latitude` - the latitude where the meter was engaged
- `dropoff_longitude` - the longitude where the meter was disengaged
- `dropoff_latitude` - the latitude where the meter was disengaged
- `store_and_fwd_flag` - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server -`Y`=store and forward; `N`=not a store and forward trip
- `trip_duration` - duration of the trip in seconds

In [ ]:
df_train.columns

In [ ]:
df_train.head(5)

#### No Null Columns

It's important to notice that the provided database has been already preprocessed and cleaned, so no null values can be found in the base.

In [ ]:
df_train.isnull().sum()

## Analyzing the whole Dataset

#### Prepare the data for analysis

Before the analysis, a few steps must be done to prepare the data. Also, the dropoff_datetime column will be dropped because it's redundant since we have the trip_duration.

In [ ]:
df_train['pickup_datetime'] = df_train['pickup_datetime'].dt.to_pydatetime()
df_test['pickup_datetime'] = df_test['pickup_datetime'].dt.to_pydatetime()
df_train.drop('dropoff_datetime', axis=1, inplace=True)

#### Trip Duration

Since trip duration is the target variable, it will be the first to be checked. After aplying the describe function, a few strange values appear, like the min and max values, 1s and 3526282s (almost 980h) respectively. Trips with a such a low or high values for duration can decrease the accuracy of our model.

In [ ]:
df_train['trip_duration'].describe()

To get rid of the outliers, lets apply the the [Interquartile Range](https://en.wikipedia.org/wiki/Interquartile_range) technique

In [ ]:
Q1 = df_train['trip_duration'].quantile(0.25)
Q3 = df_train['trip_duration'].quantile(0.75)
IQR = Q3 - Q1
df_train = df_train[~((df_train['trip_duration'] < (Q1 - 1.5 * IQR)) |(df_train['trip_duration'] > (Q3 + 1.5 * IQR)))]
df_train['trip_duration'].describe()

Even applying the IQR technique, the minimum value for a trip duration keeps very low, so values bellow 90 seconds will also be disconsidered.

In [ ]:
df_train = df_train[df_train['trip_duration'] > 1]
df_train = df_train[df_train['trip_duration'] < 7200]

In [ ]:
sns.distplot(df_train['trip_duration'], axlabel='Trip Duration')

#### Analyzing the Lat-Long Columns

To understand the distribution we must plot the values for each of the Latitude and Longitude provided. From the graph is possible to see that there are some datapoints from trips starting and finishing outside NYC.

In [ ]:
def plot_lat_long(df_data):
    fig, ax = plt.subplots(2,2,figsize=(16, 10), sharex=False, sharey = False)
    sns.distplot(df_data['pickup_latitude'], axlabel = 'Pickup Latitude',ax=ax[0,0])
    sns.distplot(df_data['pickup_longitude'], axlabel = 'Pickup_Longitude', ax=ax[0,1])
    sns.distplot(df_data['dropoff_latitude'], axlabel = 'Dropoff Latitude', ax=ax[1, 0])
    sns.distplot(df_data['dropoff_longitude'], axlabel = 'Dropoff Longitude', ax=ax[1, 1])
    plt.show()
plot_lat_long(df_train)

#### Drop Values Starting or Finishing out of New York City

Select values only within the NYC bounding Box

In [ ]:
NYC_BOUNDING_BOX = [(40.4774,-74.2589), ( 40.9176, -73.7004)]

filter_lat_long = df_train['pickup_latitude'] < NYC_BOUNDING_BOX[1][0]
filter_lat_long &= df_train['pickup_latitude'] > NYC_BOUNDING_BOX[0][0]
filter_lat_long &= df_train['pickup_longitude'] < NYC_BOUNDING_BOX[1][1]
filter_lat_long &= df_train['pickup_longitude'] > NYC_BOUNDING_BOX[0][1]

filter_lat_long &= df_train['dropoff_latitude'] < NYC_BOUNDING_BOX[1][0]
filter_lat_long &= df_train['dropoff_latitude'] > NYC_BOUNDING_BOX[0][0]
filter_lat_long &= df_train['dropoff_longitude'] < NYC_BOUNDING_BOX[1][1]
filter_lat_long &= df_train['dropoff_longitude'] > NYC_BOUNDING_BOX[0][1]

df_train = df_train[filter_lat_long]

plot_lat_long(df_train)

#### Extracting New Features: DIstance and Average Speed

Two more features will be added using the fields provided. Since we have the coorinates from the pickup and dropoff points, we can calculate the distance between points. The distance function chosen for that is the Manhattan Distance, City Block distance or [Taxicab geometry](https://en.wikipedia.org/wiki/Taxicab_geometry). The final distance is given in Kilometers

The other feature is the average speed for the whole route, for that we will use the distance obtained in the previous step and the trip duration. The average speed will be in Kilometer per hour


In [ ]:
NYC_LAT_KILOMETER_PER_DEGREE = geopy.distance.distance((40.7831, -73.9712),(41.7831, -73.9712)).kilometers
NYC_LON_KILOMETER_PER_DEGREE = geopy.distance.distance((40.7831, -73.9712),(40.7831, -72.9712)).kilometers

NYC_DEGREE_KM = 111.05938787411571

def calculate_city_block_distance(df_data):
    delta_lat = np.absolute(df_data.pickup_latitude - df_data.dropoff_latitude) * NYC_LAT_KILOMETER_PER_DEGREE    
    delta_lon = np.absolute(df_data.pickup_longitude - df_data.dropoff_longitude) * NYC_LON_KILOMETER_PER_DEGREE    
    return delta_lat + delta_lon


df_train['distance'] = calculate_city_block_distance(df_train)
df_train['avg_speed'] = df_train['distance']/(df_train['trip_duration']/3600)

### Analyzing Distance and Average Speed


In [ ]:
fig, ax = plt.subplots(1,2,figsize=(16, 10), sharex=False, sharey = False)
sns.distplot(df_train['distance'], axlabel = 'Distance',ax=ax[0])
sns.distplot(df_train['avg_speed'], axlabel = 'Average Speed',ax=ax[1])

Even filtering small values for trip duration, we still can find some values that must be removed, since it don't reflects the reality, as can be seen in the graph above, where average speeds of 800 Km/h can be seen. To remove more sporious rows, we'll remove the any value with average speeds greater than 120 Km/h.
Also distances close to 0 can be found and must be removed, for that we'll remove any values smaller than 0.250km

Even filtering the values, is possible to see that the distance distribution is skewed, for this case, we will later apply a logarithmic transformation.

In [ ]:
df_train = df_train[df_train['avg_speed'] < 100]
df_train = df_train[df_train['avg_speed'] > 1]
df_train = df_train[df_train['distance'] > .25]
df_train.drop('avg_speed', axis=1, inplace=True)

#### Analyzing Pickup Date

As discussed before, only the pickup date will be used. A few features were created from the pickup datetime, they are justified by the typical behavior that we expect to see in the data, like "Rush Hours" or less traffic at weekends and Holidays. 

To analyze that behavior the following features were created: weekdays, holidays and pickup hour

In [ ]:
df_train['pickup_date'] = df_train['pickup_datetime'].dt.date
df_train['pickup_hour'] = df_train['pickup_datetime'].dt.hour
df_train['pickup_weekday'] = df_train['pickup_datetime'].dt.day_name()

holidays = [day.date() for day in calendar().holidays(start=df_train['pickup_date'].min(), end=df_train['pickup_date'].max())]
df_train['holiday'] = df_train['pickup_date'].isin(holidays)
df_train.drop('pickup_date', axis=1, inplace=True)

#### Pickup Hour

In most of cities is expected a well defined behavior regarding populational flow within the city, this way makes a sense to analyze the average trip duration per hour of the day, as can be seen in the graph bellow.

In [ ]:
avg_trip_duration_per_pickup_hour = df_train.groupby('pickup_hour')['trip_duration'].mean()
avg_trip_duration_per_pickup_hour.plot()

#### Weekday vs Pickup Hour 

As shown in the graph bellow, different day of the week present a similar behavior, similar to the previous analysis, however the duration of each trip is different for each day of the week and in weekends the curves seen to be slightly shiffted by almost 2 hours.

In [ ]:
avg_trip_duration_per_weekday = df_train.groupby(['pickup_weekday', 'pickup_hour'])['trip_duration'].mean()
avg_trip_duration_per_weekday.unstack(level=0).plot(subplots=False)

#### Holidays

Applying the same analysis for holidays we have the graph bellow, which shows the average trip duration for a normal day and for a holiday. A similar behavior can be seen for both, however the curve for holiday is slightly shifted by a little more than 2 hours

In [ ]:
avg_trip_duration_holiday = df_train.groupby(['holiday','pickup_hour'])['trip_duration'].mean()
avg_trip_duration_holiday.unstack(level=0).plot(subplots=False)
df_train.groupby(['holiday'])['trip_duration'].mean()

#### Store and Forward

The store and forward at first look may not give any information about the trip duration, but looking the graph bellow, it becomes clear there is a difference between the two. That difference may be explained by the technologies available to drivers like navigation apps, Internet or more modern cars.

In [ ]:
avg_trip_duration_per_store_flag = df_train.groupby(['store_and_fwd_flag','pickup_hour'])['trip_duration'].mean()
avg_trip_duration_per_store_flag.unstack(level=0).plot(subplots=False)
df_train.groupby(['store_and_fwd_flag'])['trip_duration'].mean()

#### Vendor ID

A naive analysis could lead to a misconception regarding the Vendor ID as a not usefull information, but given that technology has changed the private transportation market, a good routing or driver selection algorithm or papyment method can make a great difference in the trip_duration.

In [ ]:
avg_trip_duration_per_store_flag = df_train.groupby(['vendor_id','pickup_hour'])['trip_duration'].mean()
avg_trip_duration_per_store_flag.unstack(level=0).plot(subplots=False)
df_train.groupby(['vendor_id'])['trip_duration'].mean()

#### Number of Passangers

In the same way as the previous analysis, it's worth to check the behavior of the number of passagengers transported and if it might give more infomation about the trip duration.

As can be seen bellow, the graph of number of passangers shows a similar behavior for each number of passangers, but with different values for 

In [ ]:
df_train['passenger_count'].value_counts()
df_train = df_train[df_train['passenger_count']>0]

In [ ]:
# avg_trip_duration_per_passenger_count = df_train.groupby(['passenger_count'])['trip_duration'].mean()
# avg_trip_duration_per_passenger_count.plot()
avg_trip_duration_per_passenger_count = df_train[df_train['passenger_count']>0].groupby(['passenger_count','pickup_hour'])['trip_duration'].mean()
avg_trip_duration_per_passenger_count.unstack(level=0).plot(subplots=False)
df_train.groupby(['passenger_count'])['trip_duration'].mean()

## Prepare the Data to build the models

After analyzing all the field, create more features and understanding a little more about its distribution, we have to process some of the field to prepare the data, making it more suitable to the models.

### One-hot enconding

To be properly used in supervised learning models the categorical fields independent of its type, must be transformed, since wrong information can be added to the model. One classical example of that are numerical categories where there is a logic relation between the numbers (greater than, sequences, , ratios, etc.) that might not exist in the category itself.

To make the information more

In [ ]:
df_train = pd.get_dummies(df_train, columns=['vendor_id', 'passenger_count', 
                                    'store_and_fwd_flag', 'pickup_weekday', 'pickup_hour', 'holiday'])
df_train.drop(['pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'], axis=1, inplace=True)
df_train.columns

Also, as could be seen in the analysis above, some field present a very skewed distribution and apply a logarithmic transformation gives a better representation of the data distribution

In [ ]:
df_train['trip_duration'] = np.log(df_train['trip_duration'] + 1)
df_train['distance'] = np.log(df_train['distance'] + 1)

## Training the Model

### Score Function

In the Kaggle competition there is no access to the true value of the trip duration for the test dataset and even though the score function is provided there is no way to calculate it. The only way to have the true score is subtmiting the values obtained when running the model to predict the trip duration for the Test dataset provided.

The score function chosen by the challenge is the [root mean square logarithmic error (RMSLE)](https://www.kaggle.com/c/nyc-taxi-trip-duration#evaluation), which is defined as follows:  

$$ \epsilon = \sqrt{ \frac{1}{n} \sum_{i=1}^{n} ( \log(p_i + 1) - \log(a_i + 1)  )^{2}    }  \;  $$

where $n$ is the total number of observations in the set, $p_i$ is the $i$-th trip_duration prediction, and $a_i$ is the $i$-th true value trip duration.

To train the models, we'll use the provided train dataset and split it in two sets, a Train and a Test set to evaluate the models, using it as to compare the models performance. The split will follow the proportion of 70% for the Train dataset and 30% for the Test dataset


In [ ]:
def kaggle_score(y_true_exp, y_pred_exp):
    y_pred_exp = np.exp(y_pred_exp) - 1
    y_true_exp = np.exp(y_true_exp) - 1
    e_log_square = np.square( np.log(y_pred_exp + 1) - np.log(y_true_exp + 1))
    score = np.sqrt((1/len(y_true_exp)) * np.sum(e_log_square))
    return score

#### Spliting the whole train dataset using train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

df_y_train = df_train['trip_duration']
df_X_train = df_train.drop(columns=['trip_duration'])

X_train, X_test, y_train, y_test = train_test_split(df_X_train,
                                                    df_y_train,
                                                    test_size = 0.3,
                                                    random_state = 3)

In [1]:
import pandas as pd
import numpy as np
import csv
import sklearn
from datetime import datetime, timedelta
import time
import xgboost as xgb
import scipy
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn import preprocessing



PATH_TRAIN_DATASET = './data/train.csv'
PATH_TEST_DATASET = './data/test.csv'
PATH_SAMPLE_SUMBISSION = './data/sample_submission.csv'

NYC_DEGREE_KM = 111.05938787411571
NYC_BOUNDING_BOX = [(40.4774,-74.2589), ( 40.9176, -73.7004)]

def calculate_city_block_distance(df_data):
    delta_lat = np.absolute(df_data.pickup_latitude - df_data.dropoff_latitude) * NYC_DEGREE_KM    
    delta_lon = np.absolute(df_data.pickup_longitude - df_data.dropoff_longitude) * NYC_DEGREE_KM    
    return delta_lat + delta_lon

def kaggle_score(y_true_exp, y_pred_exp):
    y_pred_exp = np.exp(y_pred_exp) - 1
    y_true_exp = np.exp(y_true_exp) - 1
    e_log_square = np.square( np.log(y_pred_exp + 1) - np.log(y_true_exp + 1))
    score = np.sqrt((1/len(y_true_exp)) * np.sum(e_log_square))
    return score

df_test = pd.read_csv(PATH_TEST_DATASET, infer_datetime_format=True, parse_dates=['pickup_datetime'],  index_col='id')
df_train = pd.read_csv(PATH_TRAIN_DATASET, infer_datetime_format=True,parse_dates=['pickup_datetime'], index_col='id')

df_train.drop('dropoff_datetime', axis=1, inplace=True)
df_train['pickup_datetime'] = df_train['pickup_datetime'].dt.to_pydatetime()
df_test['pickup_datetime'] = df_test['pickup_datetime'].dt.to_pydatetime()

Q1 = df_train['trip_duration'].quantile(0.25)
Q3 = df_train['trip_duration'].quantile(0.75)
IQR = Q3 - Q1
df_train = df_train[~((df_train['trip_duration'] < (Q1 - 1.5 * IQR)) |(df_train['trip_duration'] > (Q3 + 1.5 * IQR)))]

df_train = df_train[df_train['trip_duration'] > 1]
df_train = df_train[df_train['trip_duration'] < 7200]

filter_lat_long = df_train['pickup_latitude'] < NYC_BOUNDING_BOX[1][0]
filter_lat_long &= df_train['pickup_latitude'] > NYC_BOUNDING_BOX[0][0]
filter_lat_long &= df_train['pickup_longitude'] < NYC_BOUNDING_BOX[1][1]
filter_lat_long &= df_train['pickup_longitude'] > NYC_BOUNDING_BOX[0][1]

filter_lat_long &= df_train['dropoff_latitude'] < NYC_BOUNDING_BOX[1][0]
filter_lat_long &= df_train['dropoff_latitude'] > NYC_BOUNDING_BOX[0][0]
filter_lat_long &= df_train['dropoff_longitude'] < NYC_BOUNDING_BOX[1][1]
filter_lat_long &= df_train['dropoff_longitude'] > NYC_BOUNDING_BOX[0][1]


df_train['distance'] = calculate_city_block_distance(df_train)
df_train = df_train[df_train['distance'] > .1]
df_train['avg_speed'] = df_train['distance']/(df_train['trip_duration']/3600)
df_train = df_train[df_train['avg_speed'] < 100]
df_train = df_train[df_train['avg_speed'] > 1]

df_train.drop('avg_speed', axis=1, inplace=True)

df_train['pickup_date'] = df_train['pickup_datetime'].dt.date
df_train['pickup_hour'] = df_train['pickup_datetime'].dt.hour
df_train['pickup_weekday'] = df_train['pickup_datetime'].dt.day_name()

holidays = [day.date() for day in calendar().holidays(start=df_train['pickup_date'].min(), end=df_train['pickup_date'].max())]
df_train['holiday'] = df_train['pickup_date'].isin(holidays)
df_train.drop('pickup_date', axis=1, inplace=True)

df_train = df_train[df_train['passenger_count']>0]

df_train = pd.get_dummies(df_train, columns=['vendor_id', 'passenger_count', 
                                    'store_and_fwd_flag', 'pickup_weekday', 'pickup_hour', 'holiday'])
df_train.drop(['pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'], axis=1, inplace=True)

df_train['trip_duration'] = np.log(df_train['trip_duration'] + 1)
df_train['distance'] = np.log(df_train['distance'] + 1)

from sklearn.model_selection import train_test_split

df_y_train = df_train['trip_duration']
df_X_train = df_train.drop(columns=['trip_duration'])

X_train, X_test, y_train, y_test = train_test_split(df_X_train,
                                                    df_y_train,
                                                    test_size = 0.3,
                                                    random_state = 3)



## Model Selection

### Linear Regression

To understand better the performance and the capabilities of a model and have quick first insights about how the data is being pre processed and is being predicted. 

With that in mind, the first chosen model is [Linear Regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

In [34]:
%%time
from sklearn.linear_model import LinearRegression
model = LinearRegression()
clf = model.fit(X_train, y_train)
final_score = kaggle_score( y_test, clf.predict(X_test))
                           
print(f'Linear Regression Score: {final_score}')

Linear Regression Score: 0.4283722975834145
CPU times: user 1.1 s, sys: 384 ms, total: 1.48 s
Wall time: 1.48 s


### Lasso

Changing a little the approach, we'll use another linear regression model, using L1 regularisation. The L1 regularization limits the size of the coefficients by applying a penalty equals to the absolute value of the magnitude of coefficients

In [35]:
%%time
from sklearn.linear_model import Lasso
model = Lasso(random_state=3)
clf = model.fit(X_train, y_train)
final_score = kaggle_score( y_test, clf.predict(X_test))
                           
print(f'Lasso Score: {final_score}')

Lasso Score: 0.6736416145430251
CPU times: user 317 ms, sys: 316 ms, total: 633 ms
Wall time: 632 ms


### Ridge

Ridge is another linear regression model, but using a L2 regularization. The L2 penalty to the coefficients is equal to the square root of the coefficients magnitudes. 

In [36]:
%%time
from sklearn.linear_model import Ridge
model = Ridge(random_state=3)
clf = model.fit(X_train, y_train)
y_pred = clf.predict(X_test)
final_score = kaggle_score( y_test, y_pred)
                           
print(f'Ridge Score: {final_score}')

Ridge Score: 0.4283718167519953
CPU times: user 271 ms, sys: 248 ms, total: 519 ms
Wall time: 515 ms


### Decision Tree Regressor

Basically, the decision tree model works like a series of if-else questions for the different features presents in the dataset that leads to a understanding of the behaviour of the target variable according to how the features change.

Oftenly, decision trees might lead to the model overfitting the train dataset if no pruning if applyed to the tree.

In [37]:
%%time
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=3, max_depth=None, min_samples_split=2)
clf = model.fit(X_train, y_train)
y_pred = clf.predict(X_test)
final_score = kaggle_score( y_test, y_pred)
                           
print(f'Decision Tree Regressor Score: {final_score}')

Decision Tree Regressor Score: 0.56973663779682
CPU times: user 14.4 s, sys: 400 ms, total: 14.8 s
Wall time: 14.8 s


### Random Forest Regressor

Random Forest is an ensemble method that uses a set of slightly different decision trees to fit the model, the idea behind that is to avoid the overfitting, which is commom to decision trees, by using randomly different number of features ramdomly chosen for each decision tree, averaging their results to find a final model that is more balanced 

In [38]:
%%time
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=3)
clf = model.fit(X_train, y_train)
y_pred = clf.predict(X_test)
final_score = kaggle_score( y_test, y_pred)

print(f'Random Forest Regressor Score: {final_score}')

/home/luciano/Install/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random Forest Regressor Score: 0.45656023743551744
CPU times: user 1min 38s, sys: 584 ms, total: 1min 38s
Wall time: 1min 39s


### Gradient Boosting Regressor

Gradient Boosted trees is another ensemble method that combines multiple decision trees to compose a better and more precise final model. 

Differently from the Random Forest, the decision trees are not created in an random way, instead the trees are created in a more serial manner, with one tree trying to correct mistakes from the previous one. The algorithm uses strong pre-prunning, leading to a final model composed of shallow trees. This carachteristic contributes to a faster training that consumes less memory

In [39]:
%%time
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state=3)
clf = model.fit(X_train, y_train)
y_pred = clf.predict(X_test)
final_score = kaggle_score( y_test, y_pred)

print(f'Gradient Boosting Regressor Score: {final_score}')

Gradient Boosting Regressor Score: 0.40940629859357985
CPU times: user 2min 22s, sys: 571 ms, total: 2min 23s
Wall time: 2min 23s


### XGBoost: Extreme Gradient Boosting

XGBoost is another ensemble Boosting algorithm, it uses a set shallow decision trees to produce a more powerful final model. XGBoost is portable, efficient, scalable, accurate and flexible what makes it a odel worth trying in the challenge

In [40]:
%%time
import xgboost as xgb
model = xgb.XGBRegressor(random_state=3)
clf = model.fit(X_train, y_train)
y_pred = clf.predict(X_test)
final_score = kaggle_score( y_test, y_pred)

print(f'XGBoost Score: {final_score}')

XGBoost Score: 0.409344079071674
CPU times: user 1min 39s, sys: 804 ms, total: 1min 40s
Wall time: 1min 40s


### Adaboost Regressor

Adaboost is another ensemble method that uses a weighted combination of weak learners (models slightly better then a random guess) to produce the final result.

It's a simple iteractive method that uses a sample of the data with the same weight to all the points to train weak learners, those are used to predict the target value, which is compared to the true one. In the next iteractions the points with the bigger error receives the higher weight and the process starts again. 

The training ceases when the error functions is the same between interactions or the number o the total number of estimators is reached.

In [2]:
%%time
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor(random_state=3)
clf = model.fit(X_train, y_train)
y_pred = clf.predict(X_test)
final_score = kaggle_score( y_test, y_pred)

print(f'Gradient Boosting Regressor Score: {final_score}')

Gradient Boosting Regressor Score: 0.4398954977058956
CPU times: user 1min 40s, sys: 2.83 s, total: 1min 43s
Wall time: 1min 43s


### Scores of each model

As describe before the scoreing fucntion measures the error, thus a smaller error will mean a better score. Bellow are all the models evaluated in order from the best performing to the worst performing

1. __XGBoost:__ 0.409344079071674
1. __Gradient Boosting:__ 0.40940629859357985
1. __Ridge:__ 0.4283718167519953
1. __Linear Regression:__ 0.4283722975834145
1. __AdaBoost:__ 0.4398954977058956
1. __Random Forest:__ 0.45656023743551744
1. __Decision Tree:__ 0.56973663779682
1. __Lasso:__ 0.6736416145430251

## Fine tunning the 3 best models

#### Grid Search and Cross Validation

Cross-validation is a technique to verify how robust and general is the model when making predictions to different datasets.

In cross-validation the original data set is divided in $k$ parts of the same size, in each interaction the model to be validated is trained using $k-1$ parts and later predicts the target variable in the remaining part of the original dataset and a score is calculated with a determined score function.

After calculating the score for the configuration of train/test datasets, the dataset which was used to test becomes part of the training dataset and one of the parts that belonged to the train dataset becomes the test dataset and the process repeats until all the $k$ parts were used as test dataset. The final score for the model is the average of the scores found in each interaction

Simultaneously with the cross validation, scikit-learn uses the GridSearch. GridSearch implements an exhaustive search method through all the different combinations of Hyperparameters that the user wants to test in the model and with the help of cross validation evaluate which combination of the hyperparameters gives the best result for the dataset.


#### The best 3 models

For this section we'll only work with the 3 best performing models, that is the models with the lower values for the scores

In [3]:
best_models = {}

#### Ridge

In [4]:
%%time
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

model = Ridge(random_state=3)
scorer = make_scorer(kaggle_score, greater_is_better= False)

parameters = {
    'alpha': [0.5, 5.0, 10.0, 50.0],
    'solver': ['auto', 'lsqr', 'sag', 'svd']
}
clf = GridSearchCV(model, param_grid= parameters, scoring= scorer, verbose= 10, n_jobs= -1, cv= 3)
grid_fit = clf.fit(X_train, y_train)

best_params = grid_fit.best_params_
best_clf = grid_fit.best_estimator_
final_score = kaggle_score(y_test, best_clf.predict(X_test))
best_models['Ridge'] = {
    'final_score':final_score,
    'clf':best_clf,
    'params':best_params
}

print(f'Ridge best parameters: {best_params}')
print(f'Ridge Score: {final_score}')

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   27.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   39.1s
/home/luciano/Install/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  46 out of  48 | elapsed:  1.8min remaining:    4.6s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.8min finished


Ridge best parameters: {'alpha': 5.0, 'solver': 'svd'}
Ridge Score: 0.42837183187604305
CPU times: user 58.1 s, sys: 3.03 s, total: 1min 1s
Wall time: 1min 48s


### Gradient Boosting Regressor

In [5]:
%%time
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

model = GradientBoostingRegressor(random_state=3)
scorer = make_scorer(kaggle_score, greater_is_better= False)

parameters = {
    'max_depth': [3, 5],
    'n_estimators': [100, 200],
    'min_samples_split': [2, 6],
    'learning_rate': [0.1, 1.0]
}

clf = GridSearchCV(model, param_grid= parameters, scoring= scorer, verbose= 10, cv= 2, n_jobs=6)
grid_fit = clf.fit(X_train, y_train)

best_params = grid_fit.best_params_
best_clf = grid_fit.best_estimator_
final_score = kaggle_score(y_test, best_clf.predict(X_test))
best_models['GradientBoost'] = {
    'final_score':final_score,
    'clf':best_clf,
    'params':best_params
}

print(f'Gradient Boosting Regressor best parameters: {best_params}')
print(f'Gradient Boosting Regressor  Score: {final_score}')

Fitting 2 folds for each of 16 candidates, totalling 32 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:  3.0min
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:  5.8min
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed: 19.2min
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed: 24.8min
[Parallel(n_jobs=6)]: Done  25 out of  32 | elapsed: 31.5min remaining:  8.8min
[Parallel(n_jobs=6)]: Done  29 out of  32 | elapsed: 37.5min remaining:  3.9min
[Parallel(n_jobs=6)]: Done  32 out of  32 | elapsed: 39.6min finished


Gradient Boosting Regressor best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_split': 6, 'n_estimators': 200}
Gradient Boosting Regressor  Score: 0.39898042677040685
CPU times: user 11min 14s, sys: 2.24 s, total: 11min 16s
Wall time: 50min 8s


### XGBoost

In [6]:
%%time
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

model = xgb.XGBRegressor(random_state=3)
scorer = make_scorer(kaggle_score, greater_is_better=False)

parameters = {
    'max_depth': [5, 8, 10],
    'n_estimators': [200, 300],
    'learning_rate': [0.05, 0.1,],
    'reg_lambda': [1.0, 5] }

clf = GridSearchCV(model, param_grid= parameters, scoring= scorer, verbose= 10, cv=2, n_jobs=6)
grid_fit = clf.fit(X_train, y_train)

best_params = grid_fit.best_params_
best_clf = grid_fit.best_estimator_
final_score = kaggle_score(y_test, best_clf.predict(X_test))
best_models['XGBoost'] = {
    'final_score':final_score,
    'clf':best_clf,
    'params':best_params
}

print(f'XGBoost best parameters: {best_params}')
print(f'XGBoost Score: {final_score}')

Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:  7.0min
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed: 10.6min
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed: 36.5min
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed: 56.1min
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed: 80.3min
[Parallel(n_jobs=6)]: Done  42 out of  48 | elapsed: 109.1min remaining: 15.6min
[Parallel(n_jobs=6)]: Done  48 out of  48 | elapsed: 126.0min finished


XGBoost best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'reg_lambda': 5}
XGBoost Score: 0.39852413102654594
CPU times: user 8min 18s, sys: 3.35 s, total: 8min 22s
Wall time: 2h 13min 24s


## Predicting the Original Test Dataset and submitting to Kaggle

In [32]:
df_test = pd.read_csv(PATH_TEST_DATASET, infer_datetime_format=True, parse_dates=['pickup_datetime'],  index_col='id')
df_test['pickup_date'] = df_test['pickup_datetime'].dt.date
df_test['pickup_hour'] = df_test['pickup_datetime'].dt.hour
df_test['pickup_weekday'] = df_test['pickup_datetime'].dt.day_name()

holidays = [day.date() for day in calendar().holidays(start=df_test['pickup_date'].min(), end=df_test['pickup_date'].max())]
df_test['holiday'] = df_test['pickup_date'].isin(holidays)
df_test.drop('pickup_date', axis=1, inplace=True)
df_test['distance'] = calculate_city_block_distance(df_test)
df_test['distance'] = np.log(df_test['distance'] + 1)

df_test = pd.get_dummies(df_test, columns=['vendor_id', 'passenger_count', 
                                    'store_and_fwd_flag', 'pickup_weekday', 'pickup_hour', 'holiday'])
df_test.drop(['pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'], axis=1, inplace=True)
df_test.drop(['passenger_count_0', 'passenger_count_9'], axis=1, inplace=True)



In [21]:
import os
BASE_PATH_KAGGLE_SUBMISISON = './out'

def create_txt_file_for_submission(df_data, file_name):
    
    final_path = os.path.abspath(os.path.join(BASE_PATH_KAGGLE_SUBMISISON,file_name))
    final_path += '.txt'
    df_data.index.name = 'id'
    print(final_path)
    df_data.to_csv(final_path,
                   sep=',',
                   header=True,
                   na_rep=df_data['trip_duration'].quantile(0.5)
                  )
   

In [33]:
for model_name, model_specs in best_models.items():
    clf = sklearn.base.clone(model_specs['clf'])
    clf.fit(df_X_train, df_y_train)
    y_predict = clf.predict(df_test)
    y_true_exp = np.exp(y_predict) - 1
    df_result = pd.DataFrame(y_true_exp, columns=['trip_duration'], index=df_test.index.values)
    create_txt_file_for_submission(df_result, model_name)

/home/luciano/Personal/udacity/udacity_capstone_project/out/Ridge.txt
/home/luciano/Personal/udacity/udacity_capstone_project/out/GradientBoost.txt
/home/luciano/Personal/udacity/udacity_capstone_project/out/XGBoost.txt


**Ridge**: 0.51110

**Gradient Boost**: 0.49671

**XGBoost**: 0.49443